## k-fold cross validation

Evaluate a model trained with k-fold cross validation where each site is treated as a separate fold. Predictions from all test folds are combined and the performance is evaluated against the entire ground truth.

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import shutil
from tide.tidecv import TIDE, datasets

SPLIT = 'test'
PREDICTIONS_PATH = '/path/to/kfold_results/RUN_ID'
GT_PATH = '/path/to/dronewaste/dronewaste_v1.0.json'

### prepare ground truth and predictions

In [ ]:
# load global gt
with open(GT_PATH, 'r') as f:
    global_gt = json.load(f)

# map image filename to image id
mapping = {
    img['file_name'].split('.')[0]: img['id'] for img in global_gt['images']
}

# enumerate sites in the global gt
sites_ggt = {img['site'] for img in global_gt['images']}

# create folder for temporary site gts
tmp_path = os.path.join(PREDICTIONS_PATH, 'tmp_eval')
os.makedirs(tmp_path, exist_ok=True)


# validate prediction files and merge all predictions into one file
all_predictions = []
sites_prds = set()

# enumerate all folders (one for each fold)
folders = [
    f for f in os.listdir(PREDICTIONS_PATH)
    if os.path.isdir(os.path.join(PREDICTIONS_PATH, f))
    and f != 'tmp_eval'
]

for folder in folders:
    pred_file = os.path.join(PREDICTIONS_PATH, folder, SPLIT, 'predictions.json')
    with open(pred_file, 'r') as f:
        site_predictions = json.load(f)

    # extract site name -> e.g. "site1_100"
    curr_site = site_predictions[0]['image_id'].split('_')[0]

    # check that all predictions are from the same site
    for pred in site_predictions:
        assert pred['image_id'].split('_')[0] == curr_site, f'{pred["image_id"]} appears in predictions for site: {curr_site}'

        # convert image filename to image id
        pred['image_id'] = mapping[pred['image_id']]
        # for yolo: category id must be shifted by 1
        if 'yolo' in PREDICTIONS_PATH:
            pred['category_id'] = pred['category_id'] - 1

    sites_prds.add(curr_site)
    all_predictions.extend(site_predictions)

assert sites_prds == sites_ggt, 'Sites in predictions and ground truth are different'

print('validated predictions files')
print(f'found {len(sites_prds)} sites')

# save merged predictions
with open(os.path.join(tmp_path, 'all_predictions.json'), 'w') as f:
    json.dump(all_predictions, f)

print('merged all predictions')

### evaluate global results

In [ ]:
# load global dataset
ds = datasets.COCO(GT_PATH)
# load all predictions
preds = datasets.COCOResult(
    path=os.path.join(tmp_path, 'all_predictions.json'),
    name='preds',
)

tide = TIDE()
results = tide.evaluate_range(
    gt=ds,
    preds=preds,
    thresholds=tide.COCO_THRESHOLDS,
    mode=TIDE.BOX,
)

# extract mAP all thresholds
runs = tide.run_thresholds['preds']

metrics = {f'mAP@{int(trun.pos_thresh*100)}': trun.ap for trun in runs}
metrics['mAP@50-95'] = np.mean(list(metrics.values()))

# extract mAP per class
tide_run = tide.runs['preds']
class_aps = [0.0] * len(ds.classes)
for cl, ap in tide_run.per_classes_ap.items():
    class_aps[cl] = ap

header = ['mAP@50-95', 'mAP@50'] + [f"AP_{cls['name']}" for cls in global_gt['categories']]
values = [metrics['mAP@50-95'], metrics['mAP@50']] + class_aps

df = pd.DataFrame([values], columns=header)

csv_path = os.path.join(PREDICTIONS_PATH, f'{os.path.basename(PREDICTIONS_PATH)}_metrics.csv')
df.to_csv(csv_path, index=False)

print('global metrics saved to:', csv_path)

df

### cleanup

In [ ]:
# remove tmp directory
shutil.rmtree(tmp_path)

print('done')